In [1]:
import numpy as np
import pandas as pd

In [3]:
movies= pd.read_csv("C:/Users/Dell/Documents/Data_Science/Movie_Recommendation_System/Datasets/tmdb_5000_movies.csv")
credits= pd.read_csv("C:/Users/Dell/Documents/Data_Science/Movie_Recommendation_System/Datasets/tmdb_5000_credits.csv")

In [2]:
movies.head(2)

In [3]:
credits.head(1)['cast'].values #It return all values of cast for first movie in the list (Avatar)

In [4]:
credits.head(1)

In [7]:
movies = movies.merge(credits, on='title')# we merge movies dataset with credits datasets by using title column and assogn in movies variable

In [8]:
movies.shape # dimesion of dataset(movies)

(4809, 23)

In [5]:
movies.info()

In [23]:
#Now we are going to remove extra column which we have not required, below are the list of column that is necessary for our project
#genres, id, keywords,title,overview, tile,cast,crew  ( we have ignored all the column which we have have numerical data type mostly)

In [6]:
# We extract the above column from datasets
movies= movies[['movie_id','title','overview','genres','keywords','cast','crew']] 
movies.info() # total 6 column

In [7]:
movies.isnull().sum() # checking null value

In [8]:
movies.dropna(inplace=True) # Droping null value data
movies.isnull().sum()

In [9]:
movies.duplicated().sum()#Checking duplicate value

In [10]:
movies.iloc[0].genres # Return data for 0 row index location data for genre column

In [15]:
#Pre-Processing
#We want to remove all the puctuation (",: etc) and get only name values 
# We want to convert this dictonary into list in below format
#['Action','Adventure','fantasy','SciF']
# Below data is list of dictionary of string
#'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [16]:
# We make helper function for above task
def convert(obj):
    L=[]
    for i in obj:
        L.append(L['name'])
    return L

In [17]:
#convert('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')
# First we need to convert this string of list into list 

In [11]:
#Below module and method are able to concert the string of list into list
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [19]:
# We make helper function for above task
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [17]:
convert('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [12]:
# Now we apply the convert function on genres column to get the without puctuation data
genres_update_list = movies['genres'].apply(convert)  # Apply a function along an axis(Particular column data) of the DataFrame
genres_update_list

In [13]:
#Assign the updated genres column in movie dataset
movies['genres'] = genres_update_list 
movies.head(2)

In [22]:
# we will do for keywords column as well
movies['keywords'] = movies['keywords'].apply(convert)

In [14]:
movies.head(2)

In [107]:
#Now we want to recommend for cast cloumn on basis of top three actors,so first we need to extract the top three actors from cast column
#movies['cast'][0]

In [24]:
def convert_top3_actors(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter !=3:               # Extract top three name of cast 
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [25]:
movies['cast']=movies['cast'].apply(convert_top3_actors)

In [15]:
movies.head(2)

In [27]:
#For crew we only want to recommend on basis of Director name
#So I neeed to extract the values of that dictionary which contains director name
#movies['crew'][0]
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job'] == "Director": # via job key we find the director designation
            L.append(i['name'])    # here we append the value (name of directopr)
            break
    return L

In [28]:
Director_name_list = movies['crew'].apply(fetch_director) # It return the list of all director name for each row of data

In [29]:
movies['crew'] = Director_name_list # Assigning the value into main movies dataset

In [16]:
movies.head(2)

In [128]:
movies['overview'][0]  # It is string so we convert it into list so that we can combine this

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [173]:
# Both function convert the string into list seprated by space
#def string_to_list(obj):
#    L=[]
#    L.append(obj.split())
#    return L

In [31]:
movies["overview"] = movies["overview"].apply(lambda X:X.split())

In [17]:
movies.head(3)

In [33]:
# We need to transformation of keywords by removing the space between them because if we see "Sam Worthington" and "Sam Mendes"
#both keyword are different name but if someone search for sam then my model is getting confuse so we combined these type of keyword
def combine_keword(obj):
    L=[]
    for i in obj:
        L.append(i.replace(" ",""))
    return L

# movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])   It also work as same AS ABOVE FUNCTION IS WORKING

In [34]:
movies['genres'] = movies['genres'].apply(combine_keword)
movies['keywords'] = movies['keywords'].apply(combine_keword)
movies['cast'] = movies['cast'].apply(combine_keword)
movies['crew'] = movies['crew'].apply(combine_keword)

In [18]:
movies.head(3)

In [36]:
# now we are going to create on column and concate the columns(overview,genres,keywords,cast,crew) into single column so that
#we can apply search or filtering logic on that new column(tags)
movies['tags'] = movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [19]:
movies.head(2)

In [38]:
#now we need only three below column for filtering so we create new df
new_df= movies[['movie_id','title','tags']]

In [20]:
new_df.head(2)

In [40]:
#now we convert the list content of tags column into string
def list_to_string(obj):
    #print(" ".join(obj))
    return " ".join(obj)

#new_df['tags'].apply(lambda x:" ".join(x))    It also work as same AS ABOVE FUNCTION IS WORKING

In [21]:
new_df['tags'] = new_df['tags'].apply(list_to_string)

In [22]:
new_df.head(2)

In [23]:
# convert all keywords of tags into lower case
new_df['tags']= new_df['tags'].apply(lambda x:x.lower())

In [24]:
new_df.head(2)

In [25]:
new_df['tags'][1]

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
cv =CountVectorizer(max_features = 5000, stop_words = 'english')

In [48]:
vectors = cv.fit_transform(new_df['tags']).toarray() # transform gives sparse matrix convrting into array

In [49]:
vectors.shape

(4806, 5000)

In [50]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [26]:
len(cv.get_feature_names())

In [27]:
#It return top 5000 words(features)
cv.get_feature_names() 

In [64]:
import nltk

In [52]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [53]:
# Below are example of stemming
print(ps.stem('loved'))
print(ps.stem('loves'))
print(ps.stem('loving'))

love
love
love


In [54]:
def stemming(text):
    L= []
    for i in text.split():
        L.append(ps.stem(i))
    return " ".join(L)
    
    

In [28]:
stemming("captain barbossa, long believed to be dead, has come back to life and is headed to the edge of the earth with will turner and elizabeth swann. but nothing is quite as it seems. adventure fantasy action ocean drugabuse exoticisland eastindiatradingcompany loveofone'slife traitor shipwreck strongwoman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger johnnydepp orlandobloom keiraknightley goreverbinski")

In [29]:
new_df['tags'] = new_df['tags'].apply(stemming)

In [56]:
from sklearn.metrics.pairwise import cosine_similarity

In [57]:
similarity = cosine_similarity(vectors)

In [58]:
similarity.shape # we remove duplictaes thats why here 4806 movie vectors but in book we mentio 5000 movies

(4806, 4806)

In [30]:
similarity[0]

In [ ]:
# If someone give me movie name(eg. Avatar) then
# 1st step : We find the index of avatar in our data frame (index of Avatar = 0)
# 2nd step : Then we find the vector for that movie (vectors[0])
# 3rd step  : Then we sort the vector (vectors[0]) and return top 5 highest

In [32]:
# Below code are able to fecth the index of the movie
mov_index = new_df[new_df['title'] == "Avatar"].index[0]
mov_index

In [31]:
similarity[mov_index]

In [33]:
# Enumerate help to bind theindex number with distance so that after sorting we have index
# It gives the distance between 0th movie to each movie distance as(1st row M0 to M0 distance), (2nd row M0 to M1 distance)
#list(enumerate(similarity[0]))
sorted(list(enumerate(similarity[0])), reverse =True , key = lambda x:x[1]) # Sorting done on 1st column (index basis)

In [34]:
new_df.head(3)

In [65]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    # find top 5 movies list with index number and distance between them
    movies_list = sorted(list(enumerate(distances)), reverse =True , key = lambda x:x[1])[1:6]
    
    for i in movies_list:
        #print(i[0])# i[0] contains index 
        print(new_df.iloc[i[0]].title)# we fetch the title of movie by index postion
    
    

In [66]:
recommend("Batman Begins")

The Dark Knight
The Dark Knight Rises
Batman
Batman & Robin
Batman
